# I. Lý thuyết

1) Mạng tích chập CNN sẽ được thực hiện như thế nào?

A. Phân chia feature map thành các ma trận theo chiều channel và thực hiện tích chập giữa mỗi ma trận kernel với từng chiều.

B. Thực hiện phép nhân ma trận kernel với chuyển vị của ma trận local receptive field để thu được giá trị feature map output.

C. Tính trung bình các phân tử trên một ma trận local receptive field.

D. Di chuyển vùng local receptive field theo chiều từ trái sang phải và từ trên xuống dưới và nhân tích chập với kernel để thu được ma trận feature map.

2) Thực hiện phép nhân tích chập giữa ma trận đầu vào kích thước width và height lần lượt là $(W, H)$ với bộ lọc kích thước $(F, F)$ và bước nhảy $S$ thì thu được ma trận đầu ra với kích thước width và height $(W', H')$ là bao nhiêu? Hãy lựa chọn công thức tổng quát nhất.

A. $W' =⌊ \frac{W-F}{S} ⌋ + 1;~ H' =⌊ \frac{H-F}{S} ⌋ + 1$

B. $W' =\lceil \frac{W-F}{S} \rceil + 1;~ H' = \lceil \frac{H-F}{S} \rceil + 1$ 

C. $W' =\lfloor \frac{W-F+2P}{S} \rfloor + 1;~ H' = \lfloor \frac{H-F+2P}{S} \rfloor + 1$ với $P$ là số lượng các véc tơ 0 padding ở bên ngoài đều hai phía.

D. $W' =\lceil \frac{W-F+P}{S} \rceil + 1;~ H' = \lceil \frac{H-F+P}{S} \rceil + 1$ với $P$ là số lượng các véc tơ 0 padding ở bên ngoài đều hai phía.

3) Mạng CNN sẽ học được những gì qua các layers?

A. Tại những layers đầu mỗi channels sẽ giúp phân biệt một loại đặc trưng, của vật thể. Những đặc trưng này sau đó được trải phẳng và đưa vào mạng MLP để phân loại vật thể.

B. Những layers cuối cùng sẽ zoom vào từng chi tiết của vật thể để phân loại vật thể.

C. Tại những layers đầu mạng sẽ học được các đặc trưng chung như các edge dọc, ngang, chéo,.... Các đặc trưng chi tiết và tổng quát giúp nhận diện vật thể được học tại những layers cuối cùng.

D. Các layers đầu tiên sẽ tập trung vào các chi tiết bộ phận của vật thể.

4) Kiến trúc chung của một mạng CNN là gì?

A. _[Conv -> BatchNorm -> Activation -> Maxpooling] x n_

B. _[BatchNorm -> Conv -> Activation -> Maxpooling] x n_ 

C. _[Conv -> BatchNorm -> Maxpooling -> Activation -> Maxpooling] x n_ 

D. _[Conv -> BatchNorm -> Maxpooling -> Activation -> Maxpooling] x n -> Flatten -> [FullyConnected -> Activation] x m -> Softmax_ 

5) Thông thường đặc điểm kích thước feature map của mạng neural sẽ như thế nào ?

A. Kích thước feature map tăng dần gấp đôi qua thời gian.

B. Kích thước feature map sẽ duy trì không đổi qua thời gian.

C. Kích thước mạng sẽ giảm dần gấp đôi sau mỗi một lần downsampling và sau đó tăng dần gấp đôi sau mỗi lần upsampling để khôi phục về kích thước ảnh input.

D. Kích thước mạng thường giảm gấp đôi sau một lần downsampling và số lượng các filters tăng dần.

1) D
<br/>
2) C
<br/>
3) C
<br/>
4) D
<br/>
5) D

# II. Thực hành

6) Thực hiện xây dựng và huấn luyện một mạng CNN ngẫu nhiên trên bộ dữ liệu [Dog and Cat](https://www.kaggle.com/c/dog-vs-cat-classification/data). Lưu ý cần thực hiện [chuẩn hóa dữ liệu](https://github.com/pytorch/examples/issues/112) đối với bộ dữ liệu ImageNet trước khi huấn luyện.

7) Grid Search kiến trúc CNN dựa trên việc tổng quát hóa một thiết kế câu 6.

8) Huấn luyện lại mô hình bằng kiến trúc ResNet và MobileNet. Đánh giá chi phí tính toán và submit kết quả độ chính xác trên các ảnh thuộc folder test.

9) Thử nghiệm các phương pháp Augmentation như Flip, Rotation, Random Crop, Bright Contrast để cải thiện kết quả mô hình.

10) Lập bảng kết quả các thử nghiệm đã sử dụng.

In [ ]:
!pip install -q kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle

from google.colab import files

files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c dog-vs-cat-classification
!unzip dog-vs-cat-classification.zip

In [ ]:
import tensorflow
from  tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_folder = './train/train'
test_folder = './test/test'
pretrained_size = (224,224)
batch_size = 30

print("Getting Data...")
train_gen = ImageDataGenerator(rescale=1./255, # normalize pixel values
                             validation_split=0.3) # hold back 30% of the images for validation
test_gen = ImageDataGenerator(rescale=1./255)

print("Preparing training dataset...")
train_generator = train_gen.flow_from_directory(
    train_folder,
    shuffle=True,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

print("Preparing validation dataset...")
validation_generator =  train_gen.flow_from_directory(
    train_folder,
    shuffle=True,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

print("Preparing test dataset...")
test_generator =  test_gen.flow_from_directory(
    test_folder,
    shuffle=False,
    target_size=pretrained_size, # resize to match model expected input
    # batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

classnames = list(train_generator.class_indices.keys())
print("class names: ", classnames)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Dropout, MaxPool2D, Flatten

In [ ]:
model = Sequential([
    Input(shape=train_generator.image_shape),
    Conv2D(32, (3, 3), activation='relu'),
    Dropout(0.2),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2)),
    Flatten(),
    Dense(len(classnames), activation='softmax')
])

print(model.summary())

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
num_epochs = 5
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = num_epochs)

In [ ]:
!pip install optuna

In [ ]:
import optuna
from optuna.trial import TrialState

In [ ]:
def create_model(trial):
    n_layers = trial.suggest_int("n_layers", 2, 6)
    model = Sequential()
    for i in range(n_layers):
        num_hidden = trial.suggest_int("n_units_l{}".format(i), 32, 512, log=True)
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.6)
        model.add(Conv2D(num_hidden, (3, 3), activation='relu'))
        model.add(Dropout(p))
        model.add(BatchNormalization())
        model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(len(classnames), activation='softmax'))
    return model

In [ ]:
def create_optimizer(trial):
    kwargs = {}
    optimizer_options = ["RMSprop", "Adam", "SGD"]
    optimizer_selected = trial.suggest_categorical("optimizer", optimizer_options)
    if optimizer_selected == "RMSprop":
        kwargs["learning_rate"] = trial.suggest_float(
            "rmsprop_learning_rate", 1e-5, 1e-1, log=True
        )
        kwargs["decay"] = trial.suggest_float("rmsprop_decay", 0.85, 0.99)
        kwargs["momentum"] = trial.suggest_float("rmsprop_momentum", 1e-5, 1e-1, log=True)
    elif optimizer_selected == "Adam":
        kwargs["learning_rate"] = trial.suggest_float("adam_learning_rate", 1e-5, 1e-1, log=True)
    elif optimizer_selected == "SGD":
        kwargs["learning_rate"] = trial.suggest_float(
            "sgd_opt_learning_rate", 1e-5, 1e-1, log=True
        )
        kwargs["momentum"] = trial.suggest_float("sgd_opt_momentum", 1e-5, 1e-1, log=True)

    optimizer = getattr(tf.optimizers, optimizer_selected)(**kwargs)
    return optimizer

In [ ]:
def learn(model, optimizer, dataset, mode="eval"):
    accuracy = tf.metrics.Accuracy("accuracy", dtype=tf.float32)

    for batch, (images, labels) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = model(images, training=(mode == "train"))
            loss_value = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
            )
            if mode == "eval":
                accuracy(
                    tf.argmax(logits, axis=1, output_type=tf.int64), tf.cast(labels, tf.int64)
                )
            else:
                grads = tape.gradient(loss_value, model.trainable_variables)
                optimizer.apply_gradients(zip(grads, model.trainable_variables))

    if mode == "eval":
        return accuracy

In [ ]:
EPOCHS = 10

def objective(trial):
    # Build model and optimizer.
    model = create_model(trial)
    optimizer = create_optimizer(trial)

    # Training and validating cycle.
    with tf.device("/cpu:0"):
        for _ in range(EPOCHS):
            learn(model, optimizer, train_generator, "train")

        accuracy = learn(model, optimizer, validation_generator, "eval")

    # Return last validation accuracy.
    return accuracy.result()

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

###ResNet

In [ ]:
resnet_base_model = keras.applications.resnet.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
print(resnet_base_model.summary())

In [ ]:
from tensorflow.keras import applications
from tensorflow.keras import Model
from tensorflow.keras.layers import Flatten, Dense

# Freeze the already-trained layers in the base model
for layer in resnet_base_model.layers:
    layer.trainable = False

# Create prediction layer for classification of our images
x = resnet_base_model.output
x = Flatten()(x)
prediction_layer_resnet = Dense(len(classnames), activation='softmax')(x) 
model_resnet = Model(inputs=resnet_base_model.input, outputs=prediction_layer_resnet)

# Compile the model
model_resnet.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Now print the full model, which will include the layers of the base model plus the dense layer we added
print(model_resnet.summary())

In [ ]:
# Train the model over 5 epochs
num_epochs = 5
history_resnet = model_resnet.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = num_epochs)

In [ ]:
# Tensorflow doesn't have a built-in confusion matrix metric, so we'll use SciKit-Learn
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

print("Generating predictions from validation data...")
# Get the image and label arrays for the first batch of validation data
x_test = validation_generator[0][0]
y_test = validation_generator[0][1]

# Use the model to predict the class
class_probabilities = model_resnet.predict(x_test)

# The model returns a probability value for each class
# The one with the highest probability is the predicted class
predictions = np.argmax(class_probabilities, axis=1)

# The actual labels are hot encoded (e.g. [0 1 0], so get the one with the value 1
true_labels = np.argmax(y_test, axis=1)

# Plot the confusion matrix
cm = confusion_matrix(true_labels, predictions)
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.colorbar()
tick_marks = np.arange(len(classnames))
plt.xticks(tick_marks, classnames, rotation=85)
plt.yticks(tick_marks, classnames)
plt.xlabel("Predicted Shape")
plt.ylabel("Actual Shape")
plt.show()

###MobileNet

In [ ]:
mobilenet_base_model = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
print(resnet_base_model.summary())

In [ ]:
from tensorflow.keras import applications
from tensorflow.keras import Model
from tensorflow.keras.layers import Flatten, Dense

# Freeze the already-trained layers in the base model
for layer in mobilenet_base_model.layers:
    layer.trainable = False

# Create prediction layer for classification of our images
x = mobilenet_base_model.output
x = Dropout(0.5)(x)
x = Flatten()(x)
prediction_layer_mobilenet = Dense(len(classnames), activation='softmax')(x) 
model_mobilenet = Model(inputs=mobilenet_base_model.input, outputs=prediction_layer_mobilenet)

# Compile the model
model_mobilenet.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Now print the full model, which will include the layers of the base model plus the dense layer we added
print(model_mobilenet.summary())

In [ ]:
# Train the model over 5 epochs
num_epochs = 5
history_mobilenet = model_mobilenet.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = num_epochs)

In [ ]:
# Tensorflow doesn't have a built-in confusion matrix metric, so we'll use SciKit-Learn
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

print("Generating predictions from validation data...")
# Get the image and label arrays for the first batch of validation data
x_test = validation_generator[0][0]
y_test = validation_generator[0][1]

# Use the model to predict the class
class_probabilities = model_mobilenet.predict(x_test)

# The model returns a probability value for each class
# The one with the highest probability is the predicted class
predictions = np.argmax(class_probabilities, axis=1)

# The actual labels are hot encoded (e.g. [0 1 0], so get the one with the value 1
true_labels = np.argmax(y_test, axis=1)

# Plot the confusion matrix
cm = confusion_matrix(true_labels, predictions)
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.colorbar()
tick_marks = np.arange(len(classnames))
plt.xticks(tick_marks, classnames, rotation=85)
plt.yticks(tick_marks, classnames)
plt.xlabel("Predicted Shape")
plt.ylabel("Actual Shape")
plt.show()

In [ ]:
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

In [ ]:
train_folder = './train/train'
test_folder = './test/test'
pretrained_size = (224,224)
batch_size = 30
CROP_LENGTH = 224

# Augmentation như Flip, Rotation, Random Crop, Bright Contrast
print("Getting Data...")
train_gen = ImageDataGenerator(
    rescale=1./255, # normalize pixel values
    rotation_range=10, # rotation image
    horizontal_flip=True, # flip image
    validation_split=0.3) # hold back 30% of the images for validation
test_gen = ImageDataGenerator(rescale=1./255)

print("Preparing training dataset...")
train_generator = train_gen.flow_from_directory(
    train_folder,
    shuffle=True,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

print("Preparing validation dataset...")
validation_generator =  train_gen.flow_from_directory(
    train_folder,
    shuffle=True,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

print("Preparing test dataset...")
test_generator =  test_gen.flow_from_directory(
    test_folder,
    shuffle=False,
    target_size=pretrained_size, # resize to match model expected input
    # batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data


classnames = list(train_generator.class_indices.keys())
print("class names: ", classnames)

In [ ]:
train_crops = crop_generator(train_generator, CROP_LENGTH)
valid_crops = crop_generator(validation_generator, CROP_LENGTH)

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
mobilenet_base_model_aug = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
print(resnet_base_model.summary())

In [ ]:
# Freeze the already-trained layers in the base model
for layer in mobilenet_base_model_aug.layers:
    layer.trainable = False

inputs = tf.keras.Input(shape=(224,224,3))
x = tf.image.adjust_contrast(inputs, 2.)
x = mobilenet_base_model_aug(x, training=False)
x = Dropout(0.5)(x)
x = Flatten()(x)
prediction_layer_mobilenet = Dense(len(classnames), activation='softmax')(x) 
model_mobilenet_aug = Model(inputs, outputs=prediction_layer_mobilenet)

In [ ]:
# Compile the model
model_mobilenet_aug.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Train the model over 5 epochs
num_epochs = 5
history_mobilenet_aug = model_mobilenet_aug.fit(
    train_crops,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = valid_crops, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = num_epochs)

### Only flip

In [ ]:
train_folder = './train/train'
test_folder = './test/test'
pretrained_size = (224,224)
batch_size = 30
CROP_LENGTH = 224

# Augmentation như Flip
print("Getting Data...")
train_gen = ImageDataGenerator(
    rescale=1./255, # normalize pixel values
    horizontal_flip=True, # flip image
    validation_split=0.3) # hold back 30% of the images for validation
test_gen = ImageDataGenerator(rescale=1./255)

print("Preparing training dataset...")
train_generator = train_gen.flow_from_directory(
    train_folder,
    shuffle=True,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

print("Preparing validation dataset...")
validation_generator =  train_gen.flow_from_directory(
    train_folder,
    shuffle=True,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

print("Preparing test dataset...")
test_generator =  test_gen.flow_from_directory(
    test_folder,
    shuffle=False,
    target_size=pretrained_size, # resize to match model expected input
    # batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

classnames = list(train_generator.class_indices.keys())
print("class names: ", classnames)

In [ ]:
mobilenet_base_model_flip = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
print(mobilenet_base_model_flip.summary())

In [ ]:
from tensorflow.keras import applications
from tensorflow.keras import Model
from tensorflow.keras.layers import Flatten, Dense

# Freeze the already-trained layers in the base model
for layer in mobilenet_base_model_flip.layers:
    layer.trainable = False

# Create prediction layer for classification of our images
x = mobilenet_base_model_flip.output
x = Dropout(0.5)(x)
x = Flatten()(x)
prediction_layer_mobilenet = Dense(len(classnames), activation='softmax')(x) 
model_mobilenet_flip = Model(inputs=mobilenet_base_model_flip.input, outputs=prediction_layer_mobilenet)

# Compile the model
model_mobilenet_flip.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Now print the full model, which will include the layers of the base model plus the dense layer we added
print(model_mobilenet_flip.summary())

In [ ]:
# Compile the model
model_mobilenet_flip.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Train the model over 5 epochs
num_epochs = 5
history_mobilenet_flip = model_mobilenet_flip.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = num_epochs)

### Only Rotation

In [ ]:
train_folder = './train/train'
test_folder = './test/test'
pretrained_size = (224,224)
batch_size = 30
CROP_LENGTH = 224

# Augmentation như Rotation
print("Getting Data...")
train_gen = ImageDataGenerator(
    rescale=1./255, # normalize pixel values
    validation_split=0.3) # hold back 30% of the images for validation
test_gen = ImageDataGenerator(rescale=1./255)

print("Preparing training dataset...")
train_generator = train_gen.flow_from_directory(
    train_folder,
    shuffle=True,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

print("Preparing validation dataset...")
validation_generator =  train_gen.flow_from_directory(
    train_folder,
    shuffle=True,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

print("Preparing test dataset...")
test_generator =  test_gen.flow_from_directory(
    test_folder,
    shuffle=False,
    target_size=pretrained_size, # resize to match model expected input
    # batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

classnames = list(train_generator.class_indices.keys())
print("class names: ", classnames)

In [ ]:
mobilenet_base_model_rot = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
print(mobilenet_base_model_rot.summary())

In [ ]:
from tensorflow.keras import applications
from tensorflow.keras import Model
from tensorflow.keras.layers import Flatten, Dense

# Freeze the already-trained layers in the base model
for layer in mobilenet_base_model_rot.layers:
    layer.trainable = False

# Create prediction layer for classification of our images
x = mobilenet_base_model_rot.output
x = Dropout(0.5)(x)
x = Flatten()(x)
prediction_layer_mobilenet = Dense(len(classnames), activation='softmax')(x) 
model_mobilenet_rot = Model(inputs=mobilenet_base_model_rot.input, outputs=prediction_layer_mobilenet)

# Compile the model
model_mobilenet_rot.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Now print the full model, which will include the layers of the base model plus the dense layer we added
print(model_mobilenet_rot.summary())

In [ ]:
# Compile the model
model_mobilenet_rot.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Train the model over 5 epochs
num_epochs = 5
history_mobilenet_rot = model_mobilenet_rot.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = num_epochs)

### Only crop


In [ ]:
train_folder = './train/train'
test_folder = './test/test'
pretrained_size = (224,224)
batch_size = 30
CROP_LENGTH = 224

# Augmentation như Rotation
print("Getting Data...")
train_gen = ImageDataGenerator(
    rescale=1./255, # normalize pixel values
    rotation_range=10, # flip image
    validation_split=0.3) # hold back 30% of the images for validation
test_gen = ImageDataGenerator(rescale=1./255)

print("Preparing training dataset...")
train_generator = train_gen.flow_from_directory(
    train_folder,
    shuffle=True,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

print("Preparing validation dataset...")
validation_generator =  train_gen.flow_from_directory(
    train_folder,
    shuffle=True,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

print("Preparing test dataset...")
test_generator =  test_gen.flow_from_directory(
    test_folder,
    shuffle=False,
    target_size=pretrained_size, # resize to match model expected input
    # batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

classnames = list(train_generator.class_indices.keys())
print("class names: ", classnames)

In [ ]:
train_crops = crop_generator(train_generator, CROP_LENGTH)
valid_crops = crop_generator(validation_generator, CROP_LENGTH)

In [ ]:
mobilenet_base_model_crop = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
print(mobilenet_base_model_crop.summary())

In [ ]:
from tensorflow.keras import applications
from tensorflow.keras import Model
from tensorflow.keras.layers import Flatten, Dense

# Freeze the already-trained layers in the base model
for layer in mobilenet_base_model_crop.layers:
    layer.trainable = False

# Create prediction layer for classification of our images
x = mobilenet_base_model_crop.output
x = Dropout(0.5)(x)
x = Flatten()(x)
prediction_layer_mobilenet = Dense(len(classnames), activation='softmax')(x) 
model_mobilenet_crop = Model(inputs=mobilenet_base_model_crop.input, outputs=prediction_layer_mobilenet)

# Compile the model
model_mobilenet_crop.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Now print the full model, which will include the layers of the base model plus the dense layer we added
print(model_mobilenet_crop.summary())

In [ ]:
# Compile the model
model_mobilenet_crop.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Train the model over 5 epochs
num_epochs = 5
history_mobilenet_crop = model_mobilenet_crop.fit(
    train_crops,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = valid_crops, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = num_epochs)

### Only bright contrast


In [ ]:
train_folder = './train/train'
test_folder = './test/test'
pretrained_size = (224,224)
batch_size = 30
CROP_LENGTH = 224

# Augmentation như Rotation
print("Getting Data...")
train_gen = ImageDataGenerator(
    rescale=1./255, # normalize pixel values
    rotation_range=10, # flip image
    validation_split=0.3) # hold back 30% of the images for validation
test_gen = ImageDataGenerator(rescale=1./255)

print("Preparing training dataset...")
train_generator = train_gen.flow_from_directory(
    train_folder,
    shuffle=True,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

print("Preparing validation dataset...")
validation_generator =  train_gen.flow_from_directory(
    train_folder,
    shuffle=True,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

print("Preparing test dataset...")
test_generator =  test_gen.flow_from_directory(
    test_folder,
    shuffle=False,
    target_size=pretrained_size, # resize to match model expected input
    # batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

classnames = list(train_generator.class_indices.keys())
print("class names: ", classnames)

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
mobilenet_base_model_bc = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
print(resnet_base_model.summary())

In [ ]:
# Freeze the already-trained layers in the base model
for layer in mobilenet_base_model_bc.layers:
    layer.trainable = False

inputs = tf.keras.Input(shape=(224,224,3))
x = tf.image.adjust_contrast(inputs, 2.)
x = mobilenet_base_model_bc(x, training=False)
x = Dropout(0.5)(x)
x = Flatten()(x)
prediction_layer_mobilenet = Dense(len(classnames), activation='softmax')(x) 
model_mobilenet_bc = Model(inputs, outputs=prediction_layer_mobilenet)

In [ ]:
# Compile the model
model_mobilenet_bc.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Train the model over 5 epochs
num_epochs = 5
history_mobilenet_bc = model_mobilenet_bc.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = num_epochs)

In [ ]:
import pandas as pd
d = {'Rotation': [history_mobilenet_rot.accuracy], 'Flip': [history_mobilenet_flip.accuracy], 'Crop': [history_mobilenet_crop.accuracy], 'Bright Contrast': [history_mobilenet_bc.accuracy], 'All Augmentation': [history_mobilenet_aug.accuracy]}
pd.DataFrame(data=d, index=['Accuracy'])